<a href="https://colab.research.google.com/github/abmmatthew/dashboards/blob/master/Assignment_6_LAB10_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np   # basic numeric module in python, for array and matrix computation
import pandas as pd  # advanced numeric module, excels in data frame analysis
import matplotlib.pyplot as plt  # for data visualization
%pylab inline
import statsmodels.formula.api as smf    # for OLS regression

Populating the interactive namespace from numpy and matplotlib



Example. Income vs Education in NYC zip code areas

Model population's cumulative income against the number of individuals of different education level within the area. Higher percentage of people with higher education levels is expected to increase an average individual earning prospects
Memo:

    IncomePerCapita----measured in USD
    rest of the columns----population number under each category, e.g.
        total population over 25 years old
        holding a Bachelor's, Master's, Doctorate degree
        graduating from high school, professional school, etc.
        having less than high school, some incomplete college etc



In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/CUSP2020PUI/Data/master/IncomeEduReg.csv')
data.head()

,Zipcode,IncomePerCapita,PopOver25,LessThanHS,HighSchool,SomeCollege,Bachelor,Master,Doctorate,ProfSchool
0,10001,77512.0,16328,1389,1665,2075,6061,3412,519,1207
1,10002,26905.0,60932,21170,12718,8532,12721,4001,641,1149
2,10003,79088.0,41182,1499,2810,4516,17958,9094,1626,3679
3,10004,98020.0,2279,29,87,305,984,550,86,238
4,10005,99633.0,5954,133,103,454,2745,1637,219,663


We will be looking to model the total income of the zip code as

where is the number of people with certain education and the weight coefficient could be interpreted as the average income per capita within this population group. No intercept as we do not expect zero population to have nonzero income

In [4]:
data.dropna(inplace = True)  #drop rows with NANs

In [5]:
data['Income'] = data['IncomePerCapita'] * data['PopOver25'] #calculate total income of the population over 25

In [6]:
#first run a regression again total topulation to define the overall average income per capita
lm2 = smf.ols(formula = 'Income ~ PopOver25-1', data = data).fit()
print(lm2.summary())

                                 OLS Regression Results                                
Dep. Variable:                 Income   R-squared (uncentered):                   0.658
Model:                            OLS   Adj. R-squared (uncentered):              0.657
Method:                 Least Squares   F-statistic:                              347.0
Date:                Mon, 09 Dec 2024   Prob (F-statistic):                    7.54e-44
Time:                        16:58:43   Log-Likelihood:                         -3976.7
No. Observations:                 181   AIC:                                      7955.
Df Residuals:                     180   BIC:                                      7959.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

And now run the regression for all the education categories

In [7]:
#use this operator to generate the regression formulae
' + '.join(data.columns[3:10])

'LessThanHS + HighSchool + SomeCollege + Bachelor + Master + Doctorate + ProfSchool'

In [8]:
lm2 = smf.ols(formula = 'Income ~ ' + ' + '.join(data.columns[3:10])+'-1', data = data).fit()
print(lm2.summary())

                                 OLS Regression Results                                
Dep. Variable:                 Income   R-squared (uncentered):                   0.971
Model:                            OLS   Adj. R-squared (uncentered):              0.970
Method:                 Least Squares   F-statistic:                              840.7
Date:                Mon, 09 Dec 2024   Prob (F-statistic):                   1.50e-130
Time:                        16:59:30   Log-Likelihood:                         -3752.6
No. Observations:                 181   AIC:                                      7519.
Df Residuals:                     174   BIC:                                      7542.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

Only two regressors seem significant according to p-value. Also the confidence intervals are pretty broad not allowing to reliably assess the income per capita for each category of residents. This is likely because of the strong internal correlations between the regressors, especially among Bachelor,Master,Doctor and Prof.School

In [9]:
data.corr()

,Zipcode,IncomePerCapita,PopOver25,LessThanHS,HighSchool,SomeCollege,Bachelor,Master,Doctorate,ProfSchool,Income
Zipcode,1.000000,-0.490675,0.089223,0.138386,0.298570,0.233799,-0.136994,-0.224106,-0.361479,-0.340607,-0.326244
IncomePerCapita,-0.490675,1.000000,-0.286507,-0.530034,-0.559255,-0.500068,0.262061,0.385013,0.447516,0.571962,0.542535
PopOver25,0.089223,-0.286507,1.000000,0.747408,0.833786,0.876240,0.691929,0.516295,0.350869,0.313306,0.470183
LessThanHS,0.138386,-0.530034,0.747408,1.000000,0.779932,0.693017,0.146826,-0.056804,-0.116706,-0.189604,-0.061991
HighSchool,0.298570,-0.559255,0.833786,0.779932,1.000000,0.918826,0.244020,0.028185,-0.135187,-0.187038,0.005785
SomeCollege,0.233799,-0.500068,0.876240,0.693017,0.918826,1.000000,0.389975,0.200074,0.014369,-0.042483,0.146218
Bachelor,-0.136994,0.262061,0.691929,0.146826,0.244020,0.389975,1.000000,0.919023,0.769090,0.810396,0.873868
Master,-0.224106,0.385013,0.516295,-0.056804,0.028185,0.200074,0.919023,1.000000,0.904758,0.913308,0.912174
Doctorate,-0.361479,0.447516,0.350869,-0.116706,-0.135187,0.014369,0.769090,0.904758,1.000000,0.889662,0.839830
ProfSchool,-0.340607,0.571962,0.313306,-0.189604,-0.187038,-0.042483,0.810396,0.913308,0.889662,1.000000,0.943770


In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
%pylab inline

# Load the data
data = pd.read_csv('https://raw.githubusercontent.com/CUSP2020PUI/Data/master/IncomeEduReg.csv')
data.dropna(inplace=True)  # Drop rows with missing values
data['Income'] = data['IncomePerCapita'] * data['PopOver25']  # Compute total income

# Create 'Other' category from available columns
data['Other'] = data['LessThanHS'] + data['HighSchool'] + data['SomeCollege']

# Perform linear regression
formula = 'Income ~ Bachelor + Advanced + Other - 1'
data['Advanced'] = data['Master'] + data['Doctorate'] + data['ProfSchool']
lm = smf.ols(formula=formula, data=data).fit()
print(lm.summary())

# Check if all regressors are statistically significant (p < 0.05)
print("\nAre all regressors statistically significant (p<0.05)?")
p_values = lm.pvalues
significant_regressors = p_values[p_values < 0.05]
if len(significant_regressors) == len(p_values):
    print("Yes, all regressors are statistically significant.")
else:
    print("No, not all regressors are statistically significant.")
    print("Significant regressors:", significant_regressors)

# Compute confidence intervals and determine the highest income per capita group
conf_intervals = lm.conf_int()
print("\nConfidence Intervals:")
print(conf_intervals)

highest_income_group = None
highest_income_lower_bound = -float('inf')

for group in ['Bachelor', 'Advanced', 'Other']:
    lower_bound = conf_intervals.loc[group, 0]
    if lower_bound > highest_income_lower_bound:
        highest_income_lower_bound = lower_bound
        highest_income_group = group

print(f"\nBased on the lower bound of the confidence interval, the group with the highest income per capita is: {highest_income_group}")

Populating the interactive namespace from numpy and matplotlib
                                 OLS Regression Results                                
Dep. Variable:                 Income   R-squared (uncentered):                   0.949
Model:                            OLS   Adj. R-squared (uncentered):              0.948
Method:                 Least Squares   F-statistic:                              1106.
Date:                Mon, 09 Dec 2024   Prob (F-statistic):                   8.50e-115
Time:                        17:09:52   Log-Likelihood:                         -3804.4
No. Observations:                 181   AIC:                                      7615.
Df Residuals:                     178   BIC:                                      7624.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|   